In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -U pyYAML
!pip install pycm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import yaml
import matplotlib.pyplot as plt
from pycm import *
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

from keras.layers import Dense, Flatten, AveragePooling2D, Dropout, MaxPool2D
from tensorflow.keras.optimizers import Adam
from keras.applications.xception import Xception
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.models import Model
from keras.layers import Dense, MaxPool2D, Conv2D


In [3]:
cfg = yaml.full_load(open("/content/drive/MyDrive/FP/config.yml", 'r'))

In [4]:
train_data = pd.read_csv(cfg['PATHS']['TRAIN_SET'])
test_data = pd.read_csv(cfg['PATHS']['TEST_SET'])
valid_data = pd.read_csv(cfg['PATHS']['VALID_SET'])

In [5]:
train_data_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input, 
    zoom_range= 0.2, 
    horizontal_flip= True, 
    shear_range= 0.2,
)


In [6]:
train_generator = train_data_gen.flow_from_dataframe(
    train_data,
    x_col='filename',
    y_col='category_str',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=128
)

Found 2199 validated image filenames belonging to 3 classes.


In [7]:
test_data_gen = ImageDataGenerator(preprocessing_function= preprocess_input  )

test_generator = test_data_gen.flow_from_dataframe(
    test_data,
    x_col='filename',
    y_col='category_str',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=128,
    shuffle = False

)

Found 733 validated image filenames belonging to 3 classes.


In [8]:
valid_data_gen = ImageDataGenerator(preprocessing_function= preprocess_input )

valid_generator = valid_data_gen.flow_from_dataframe(
    valid_data,
    x_col='filename',
    y_col='category_str',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=128
)

Found 733 validated image filenames belonging to 3 classes.


In [ ]:
vgg = VGG16( input_shape=(224,224,3), include_top= False)

for layer in vgg.layers:           # Dont Train the parameters again 
  layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(units=3 , activation='sigmoid', name = 'predictions' )(x)

model = Model(vgg.input, x)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only= True)

hist = model.fit_generator(train_generator, steps_per_epoch= 10, epochs= 50, validation_data= valid_generator , validation_steps= 32, callbacks=[es,mc])

### Statistics of the Model

In [9]:
from keras.models import load_model
model = load_model("/content/drive/MyDrive/FP/models/vgg_model.h5")
# model.save(cfg['PATHS']['VGG'])

In [10]:
acc = model.evaluate_generator(generator= test_generator)[1] 

print(f"The accuracy of your model is = {acc} %")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


The accuracy of your model is = 0.9549795389175415 %


In [11]:
y_pred = model.predict(test_generator,verbose =0)

In [12]:
y_pred = np.argmax(y_pred,axis =-1)
y_pred

y_pred.shape

(733,)

In [13]:
test_labels = test_generator.labels
cm = ConfusionMatrix(actual_vector=test_labels, predict_vector=y_pred)

In [14]:
cm.print_matrix()

Predict   0         1         2         
Actual
0         215       3         0         

1         20        224       2         

2         5         53        211       




In [15]:
cm.stat(summary=True)

Overall Statistics : 

ACC Macro                                                         0.92451
F1 Macro                                                          0.8887
FPR Macro                                                         0.05595
Kappa                                                             0.83043
Overall ACC                                                       0.88677
PPV Macro                                                         0.89548
SOA1(Landis & Koch)                                               Almost Perfect
TPR Macro                                                         0.89373
Zero-one Loss                                                     83

Class Statistics :

Classes                                                           0             1             2             
ACC(Accuracy)                                                     0.9618        0.89359       0.91814       
AUC(Area under the ROC curve)                                     0.968

In [20]:
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94       218
           1       0.80      0.91      0.85       246
           2       0.99      0.78      0.88       269

    accuracy                           0.89       733
   macro avg       0.90      0.89      0.89       733
weighted avg       0.90      0.89      0.89       733

